In [1]:
import pandas as pd
import string
import re
import string
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer 

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
import pickle_functions as pf

### Load and split data

Read in test.csv and train.csv

In [14]:
train = pf.read_pickle(bucket_name='advancedml-koch-mathur-hinkson', filename='sub_train_df10_preprocessed')

In [15]:
train.shape

(100000, 60)

In [16]:
drop_cols = ['split', 'cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter', 'cleaned_lancaster', 'bigrams_unstemmed',
       'perc_upper', 'num_exclam', 'num_words', 'perc_stopwords',
       'num_upper_words']

In [17]:
train = train.drop(drop_cols, axis = 1)

In [18]:
train.shape

(100000, 49)

In [19]:
train.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count',
       'cleaned_w_stopwords_str', 'cleaned_no_stem_str', 'cleaned_porter_str',
       'cleaned_lancaster_str'],
      dtype='object')

Create a new column called "toxicity_category" in the train data frame categorizing comments as toxic ("1") or non-toxic ("0").

In [20]:
train['toxicity_category'] = train.target.apply(lambda x: 1 if x > 0.5 else 0)

Split train.csv into training (80%) and validation sets (20%).

In [22]:
# https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
msk = np.random.rand(len(train)) < 0.8
train_set = train[msk]
validation_set = train[~msk]

In [23]:
print(train_set.toxicity_category.value_counts())

0    75292
1     4537
Name: toxicity_category, dtype: int64


In [24]:
print(validation_set.toxicity_category.value_counts())

0    19088
1     1083
Name: toxicity_category, dtype: int64


Create small sample ("train_sample1") from the train_set on which to run models.  Ensure that samples are iid by replacing after each draw.

In [9]:
# train_sample = train_set.sample(frac=0.2, replace=True)

In [25]:
print(train_set.toxicity_category.value_counts())

0    75292
1     4537
Name: toxicity_category, dtype: int64


In [2]:
# train_set = pd.read_csv('processed_sample_20_perc.csv')
# train_set = train_set.sample(frac=0.5)

In [26]:
toxic = train_set[train_set.toxicity_category == 1]
nontoxic = train_set[train_set.toxicity_category == 0]

In [27]:
train_set.shape, toxic.shape, nontoxic.shape

((79829, 50), (4537, 50), (75292, 50))

Reshaping the dataset to be include an equal number of toxic and nontoxic samples

In [28]:
quarter = len(toxic)

In [29]:
random_df = train_set.sample(quarter*4)

In [30]:
prepared_25 = toxic.append(nontoxic.sample(len(toxic)*3))
prepared_25 = prepared_25.sample(frac=1).reset_index(drop=True)
print(prepared_25.toxicity_category.value_counts())

prepared_50 = toxic.append(toxic).append(nontoxic.sample(len(toxic)*2))
prepared_50 = prepared_50.sample(frac=1).reset_index(drop=True)
print(prepared_50.toxicity_category.value_counts())

prepared_75 = toxic.append(toxic).append(toxic).append(nontoxic.sample(len(toxic)))
prepared_75 = prepared_75.sample(frac=1).reset_index(drop=True)
print(prepared_75.toxicity_category.value_counts())


0    13611
1     4537
Name: toxicity_category, dtype: int64
1    9074
0    9074
Name: toxicity_category, dtype: int64
1    13611
0     4537
Name: toxicity_category, dtype: int64


In [31]:
def run_model(model_df, train_perc=.80,  model_type = "SVM", 
             see_inside=False, comments="comment_text",
             target='toxicity_category'):
    '''
    This function runs a single machine learning model as per the specified parameters.
    
    Input(s):
        model_df: source data frame
        train_perc: percentage that should be used for training set
        addtl_feats: (list) list of non text columns to include
        model_type: which machine learning model to use
        see_inside: returns the intermediate tokenized and vectorized arrays
        comments: source column for text data
        target: source column for y values
        
    Output(s):
    
    '''
    
    train_start = 0
    train_end = round(model_df.shape[0]*train_perc) 

    test_start = train_end
    test_end = model_df.shape[0]
    
    X_all = model_df[comments].values
    y_all = model_df[target].values

    # calculating frequencies
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    fitted_vectorizer=tfidf_vectorizer.fit(model_df[comments].values.astype('U'))
    X_all_tfidf =  fitted_vectorizer.transform(model_df[comments].values.astype('U'))
    
    
    X_train = X_all_tfidf[train_start:train_end]
    y_train = model_df[train_start:train_end][target].values
    y_train=y_train.astype('int')
    

    X_test = X_all_tfidf[test_start:test_end]
    y_test = model_df[test_start:test_end][target].values
    print("fitting model now")
    model_dict = {}
    model_dict["MultiNB"] = MultinomialNB()
    model_dict["GaussNB"] = GaussianNB()
    model_dict['SVM'] = svm.SVC(kernel='linear', probability=True, random_state=1008)
    model_dict["LR"] = LogisticRegression(penalty="l1",C=1e5)
        
    clf = model_dict[model_type].fit(X_train, y_train)
    
    predicted = clf.predict(X_test)
    
    output = model_df[test_start:test_end]
    output['predicted'] = predicted
    output['y_test'] = y_test
    output['accuracy'] = output.predicted == output.y_test
    
    if see_inside == True:
        return clf, output, X_all_counts, X_all_tfidf
    else:
        return clf, output


In [32]:
def get_metrics(output, should_print=True, round_to=3):
    metrics = {}
    targets = output[output.y_test == 1]
    nontargets = output[output.y_test == 0]
    
    dfs = [output, targets, nontargets]
    labels = ["Overall", "Target", "Non-Target"]
    
    for i in range(len(dfs)):

        df, label = dfs[i], labels[i]
        if label == "Non-Target":
            pos_label = 0
        else:
            pos_label = 1
        
        metrics[label] = {}
        
        
        accuracy = round(accuracy_score(df.y_test, df.predicted), round_to)
        metrics[label]['Accuracy'] = accuracy
        
        precision = round(precision_score(df.y_test, df.predicted, pos_label=pos_label), round_to)
        metrics[label]['Precision'] = precision

        recall = round(recall_score(df.y_test, df.predicted, pos_label=pos_label), round_to)
        metrics[label]['Recall'] = recall
        
        f1 = round(f1_score(df.y_test, df.predicted, pos_label=pos_label), round_to)
        metrics[label]['F1'] = f1

        if label == "Overall":
            roc_auc = round(roc_auc_score(df.y_test, df.predicted), round_to)
            metrics[label]['ROC_AUC'] = roc_auc
            
        if should_print == True:
            print("{} Accuracy: {}".format(label, accuracy))
            print("{} Precision: {}".format(label, precision))
            print("{} Recall: {}".format(label, recall))
            print("{} F1 Score: {}".format(label, f1))
            if label == "Overall":
                print("ROC_AUC: {}".format(label, roc_auc))
            print()
            
    return metrics
   

### Highlighted Model

In [ ]:
clf, output = run_model(prepared_50, comments = "cleaned_no_stem")

In [ ]:
get_metrics(output)

### SVM

In [33]:
best_metric = 0
metric_dict = ''
model_factors = []

SUBSET_OF_INTEREST = "Target"
METRIC_OF_INTEREST = "F1"

dfs = [random_df,  prepared_50]
label = ["random_df", "prepared_50"]

mn = 0

for i in range(len(dfs)):
    for text in ['cleaned_w_stopwords_str', 'cleaned_no_stem_str', 'cleaned_porter_str',
       'cleaned_lancaster_str']:

        factors = [label[i], text]
        mn += 1
        print("{}. {}".format(mn, datetime.datetime.now()))
        print(factors)

        clf, output = run_model(dfs[i], comments = text)
        metrics = get_metrics(output, should_print=False)
        metric_of_interest = metrics[SUBSET_OF_INTEREST][METRIC_OF_INTEREST]
        
        print("Overall Accuracy: {}, Target Accuracy: {}, Non-Target Accuracy: {}".format(metrics["Overall"]["Accuracy"], metrics["Target"]["Accuracy"], metrics["Non-Target"]["Accuracy"]))
        print() 
        
        if (metric_of_interest > best_metric) and metric_of_interest < 0.95:
            best_metric = metric_of_interest
            
            model_factors = factors
            metric_dict = metrics

1. 2019-05-29 19:49:51.041184
['random_df', 'cleaned_w_stopwords_str']
fitting model now
Overall Accuracy: 0.954, Target Accuracy: 0.214, Non-Target Accuracy: 0.997

2. 2019-05-29 19:51:31.170383
['random_df', 'cleaned_no_stem_str']
fitting model now
Overall Accuracy: 0.955, Target Accuracy: 0.244, Non-Target Accuracy: 0.997

3. 2019-05-29 19:52:46.985737
['random_df', 'cleaned_porter_str']
fitting model now
Overall Accuracy: 0.956, Target Accuracy: 0.284, Non-Target Accuracy: 0.996

4. 2019-05-29 19:53:50.908204
['random_df', 'cleaned_lancaster_str']
fitting model now
Overall Accuracy: 0.957, Target Accuracy: 0.279, Non-Target Accuracy: 0.997

5. 2019-05-29 19:54:53.467464
['prepared_50', 'cleaned_w_stopwords_str']
fitting model now
Overall Accuracy: 0.89, Target Accuracy: 0.884, Non-Target Accuracy: 0.896

6. 2019-05-29 19:59:36.379812
['prepared_50', 'cleaned_no_stem_str']
fitting model now
Overall Accuracy: 0.892, Target Accuracy: 0.884, Non-Target Accuracy: 0.9

7. 2019-05-29 20:0

In [34]:
model_factors, best_metric

(['prepared_50', 'cleaned_w_stopwords_str'], 0.939)

In [35]:
metric_dict

{'Overall': {'Accuracy': 0.89,
  'Precision': 0.895,
  'Recall': 0.884,
  'F1': 0.889,
  'ROC_AUC': 0.89},
 'Target': {'Accuracy': 0.884, 'Precision': 1.0, 'Recall': 0.884, 'F1': 0.939},
 'Non-Target': {'Accuracy': 0.896,
  'Precision': 1.0,
  'Recall': 0.896,
  'F1': 0.945}}